In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


The formated text from an image into a single sequene is fed into a two-layer bidirectional LSTM to produce a classification label (4 key information category )

BiLSTM means bidirectional LSTM, which means the signal propagates backward as well as forward in time.

Made in pytorch


In [ ]:
!pip install colorama;

In [ ]:
import torch
from torch import nn,optim
import json
import random
import os
from os import path
from string import ascii_uppercase, digits, punctuation
#import colorama
import numpy
import regex
# from colorama import Fore
from torch.utils import data
from difflib import SequenceMatcher

In [ ]:
class TextBox(object):
    def __init__(self, line):
        
        line_split = line.strip().split(",", maxsplit=8)
        self.xspan = (int(line_split[0]), int(line_split[4]))
        self.yspan = (int(line_split[1]), int(line_split[5]))
        self.x = (self.xspan[0] + self.xspan[1]) / 2
        self.y = (self.yspan[0] + self.yspan[1]) / 2
        self.text = line_split[8]

    def __repr__(self):
        return self.text


class TextLine(object):
    def __init__(self, text_box=None):
        if isinstance(text_box, TextBox):
            self.text = [text_box.text]
            self.xs = [text_box.x]
            self.y = text_box.y
            self.yspan = text_box.yspan
        else:
            self.text = []
            self.xs = []
            self.yspan = None

    def insert(self, text_box):
        if not (
            (text_box.yspan[0] < self.y < text_box.yspan[1])
            and (self.yspan[0] < text_box.y < self.yspan[1])
        ):
            raise ValueError

        try:
            at = next(i for i, v in enumerate(self.xs) if v > text_box.x)
            self.text.insert(at, text_box.text)
            self.xs.insert(at, text_box.x)
        except StopIteration:
            self.text.append(text_box.text)
            self.xs.append(text_box.x)

        self.y = text_box.y
        self.yspan = text_box.yspan

    def __str__(self):
        return "\t".join(self.text)

    def __repr__(self):
        if self.yspan is None:
            repr_yspan = "[    ,    ] "
        else:
            repr_yspan = "[{:4d},{:4d}] ".format(self.yspan[0], self.yspan[1])

        repr_text = "\t".join(self.text)

        return repr_yspan + repr_text

In [ ]:
def sort_text(txt_file):

    with open(txt_file, "r",encoding='cp1252') as txt_opened:
        content = sorted([TextBox(line.strip()) for line in txt_opened if not line.isspace()], key=lambda box: box.y)

    text_lines = [TextLine(content[0])]
    for box in content[1:]:
        try:
            text_lines[-1].insert(box)
        except ValueError:
            text_lines.append(TextLine(box))

    return "\n".join([str(text_line) for text_line in text_lines])

In [ ]:
infer_data_path = r'/content/gdrive/My Drive/infer/'
infer_dict_path = os.path.join(infer_data_path,'infer_dict.pth') # sample 
def create_infer_data():
    keys = sorted(
        path.splitext(f.name)[0]
        for f in os.scandir(infer_data_path)
    )

    files = [os.path.join(infer_data_path,s+'.txt') for s in keys if '.pth' not in s]

    infer_dict = {}
    for k, f in zip(keys, files):
        print(f)
        print(sort_text(f))
        infer_dict[k] = sort_text(f)
    torch.save(infer_dict, infer_dict_path)

import os; os.listdir(infer_data_path)
os.remove(infer_dict_path)
create_infer_data()

/content/gdrive/My Drive/infer/Copy of X51009453804.txt
3180303
lian hing stationery sdn bhd
(162761-m)
no.32 & 33 jalan sr 1/9 seksyen 9
taman serdang raya
43300 seri kembangan selangor
darul ehsan
gst id : 002139201536
tax invoice
27/03/2018	no : cs-20243
qty	tax	rm
f/castell 187057- 75 tack-lt	2	sr	12.00
75g- white (new) @ 5.6600
total amt incl. gst @ 6% :	12.00
rounding adjustment :
total amt payable :	12.00
paid amount :	20.00
change :	8.00
total qty tender :	2
gst summary	amount	tax
(rm)	(rm)
sr @ a	11.32	0.68
total	11.32	0.68
thank you
for any enquiry please contact us:
/content/gdrive/My Drive/infer/Copy of X51009453804.txt
3180303
lian hing stationery sdn bhd
(162761-m)
no.32 & 33 jalan sr 1/9 seksyen 9
taman serdang raya
43300 seri kembangan selangor
darul ehsan
gst id : 002139201536
tax invoice
27/03/2018	no : cs-20243
qty	tax	rm
f/castell 187057- 75 tack-lt	2	sr	12.00
75g- white (new) @ 5.6600
total amt incl. gst @ 6% :	12.00
rounding adjustment :
total amt payable :	12.00


In [ ]:
class MyModel0(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers=2, bidirectional=True)
        self.linear = nn.Linear(hidden_size * 2, 5)

    def forward(self, inpt):
        embedded = self.embed(inpt)
        feature, _ = self.lstm(embedded)
        oupt = self.linear(feature)

        return oupt

In [ ]:
VOCAB = ascii_uppercase + digits + punctuation + " \t\n" + "£"+ "¬"


In [ ]:
data_loc = r'/content/gdrive/My Drive/sroie_task3/data'
data_dict = os.path.join(data_loc,'data_dict4.pth') # data for retraining
model_loc =os.path.join(data_loc,'model.pth') # retraining

In [ ]:

class MyDataset(data.Dataset):
    def __init__(
        self, 
        dict_path=data_dict, 
        device="cpu", 
        val_size=76, 
        test_path=None
        ):
      
        if dict_path is None:
            self.val_dict = {}
            self.train_dict = {}
        else:
            data_items = list(torch.load(dict_path).items())
            random.shuffle(data_items)

            self.val_dict = dict(data_items[:val_size])
            self.train_dict = dict(data_items[val_size:])

        if test_path is None:
            self.test_dict = {}
        else:
            self.test_dict = torch.load(test_path)

        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    def get_test_data(self, key):
        text = self.test_dict[key].upper()
        text_tensor = torch.zeros(len(text), 1, dtype=torch.long)
        text_tensor[:, 0] = torch.LongTensor([VOCAB.find(c) for c in text])

        return text_tensor.to(self.device)

In [ ]:
def pred_to_dict(text, pred, prob):
    res = {"company": ("", 0), "date": ("", 0), "address": ("", 0), "total": ("", 0)}
    keys = list(res.keys())

    seps = [0] + (numpy.nonzero(numpy.diff(pred))[0] + 1).tolist() + [len(pred)]
    for i in range(len(seps) - 1):
        pred_class = pred[seps[i]] - 1
        if pred_class == -1:
            continue

        new_key = keys[pred_class]
        new_prob = prob[seps[i] : seps[i + 1]].max()
        if new_prob > res[new_key][1]:
            res[new_key] = (text[seps[i] : seps[i + 1]], new_prob)

    return {k: regex.sub(r"[\t\n]", " ", v[0].strip()) for k, v in res.items()}


In [ ]:
def test(hidden_size=256):

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    model = MyModel0(len(VOCAB), 16, hidden_size).to(device)
    dataset = MyDataset(None, device, test_path=infer_dict_path)

    model.load_state_dict(torch.load(model_loc))

    model.eval()
    with torch.no_grad():
        for key in dataset.test_dict.keys():
            text_tensor = dataset.get_test_data(key)
            print(key)
            
            oupt = model(text_tensor)
            prob = torch.nn.functional.softmax(oupt, dim=2)
            prob, pred = torch.max(prob, dim=2)

            prob = prob.squeeze().cpu().numpy()
            pred = pred.squeeze().cpu().numpy()

            real_text = dataset.test_dict[key]
            result = pred_to_dict(real_text, pred, prob)

            with open(infer_data_path + key + ".json", "w", encoding="utf-8") as json_opened:
                json.dump(result, json_opened, indent=4)
            print(result)

test()

Copy of X51009453804
{'company': 'lian hing stationery sdn bhd', 'date': '27/03/2018', 'address': 'no.32 & 33 jalan sr 1/9 seksyen 9 taman serdang raya 43300 seri kembangan selangor darul ehsan', 'total': '12.00'}
Copy of file1
{'company': '', 'date': '18/01/2018', 'address': '', 'total': '4.90'}
X51005361883
{'company': 'teo heng statiowmery b-80bmks', 'date': '18/01/2018', 'address': 'no. 53 jalan bmsar 45600 batang barjuntar selangor darul ehsan', 'total': '4.90'}
